In [11]:
from __future__ import division, absolute_import, print_function

from six.moves import urllib, xrange

import tensorflow as tf
import numpy as np
import collections, math, os, random, zipfile
import glob,os
import nltk
import re
import string

In [12]:
os.chdir("/Users/Nikhil/Documents/Courses/Knowledge Discovery in Databases/Final Project/diabetes 2/")

files = []
words = []
punkt = nltk.data.load('tokenizers/punkt/english.pickle')
htmlTagsRegex = r"<([A-Z][A-Z0-9]*)\b[^>]*>(.*?)<\/\1>"
brTagsRegex = r"<([A-Z][A-Z0-9]*)\b[^>]*>"
links = r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+'
speacialChars = [":"]
for file in glob.glob("*.txt"):
    file = open(file)
    files.append(file)
    fileData = file.read()
    #re.sub(htmlTagsRegex,'',fileData)
    #re.sub(brTagsRegex,'',fileData)
    fileData = re.sub(links,'',fileData)
    fileData = re.sub(r'\d+', '', fileData)   
    #fileData = re.sub('[^0-9-_*.]', '', fileData)
    #fileData.translate(None,string.punctuation)
    sentences = nltk.sent_tokenize(fileData)
    for sentance in sentences:
        words.extend(nltk.word_tokenize(sentance))
    file.close()

print("Number of Words in corpus: {0}".format(len(words)))

Number of Words in corpus: 4888033


In [13]:
stopwords = nltk.corpus.stopwords.words("english")
excludeList = ['ii',"``","''"]
exclude = set(string.punctuation)
words = [ch for ch in words if ch not in exclude]
words = [ch for ch in words if ch not in excludeList]
words = [word.lower() for word in words]
words = [word for word in words if word not in stopwords]
print("Number of Words in corpus: {0}".format(len(words)))

Number of Words in corpus: 2648067


In [14]:
# build the dictionary and replace rare words with the "UNK" token.
vocabulary_size = 150000

def build_dataset(words):
    # create counts list, set counts for "UNK" token to -1 (undefined)
    count = [['UNK', -1]]
    # add counts of the 49,999 most common tokens in 'words'
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    # create the dictionary data structure
    dictionary = {}
    # give a unique integer ID to each token in the dictionary
    for word, _ in count:
        dictionary[word] = len(dictionary)
    # create a list data structure for the data
    data = []
    # keep track of the number of "UNK" token occurrences
    unk_count = 0
    # for each word in our list of words
    for word in words:
        # if its in the dictionary, get its index
        if word in dictionary:
            index = dictionary[word]
        # otherwise, set the index equal to zero (index of "UNK") and increment the "UNK" count
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        # append its index to the 'data' list structure
        data.append(index)
    # set the count of "UNK" in the 'count' data structure
    count[0][1] = unk_count
    # invert the dictionary; it becomes (index, word) key-value pairs
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    # return the data (indices), counts, dictionary, and inverted dictionary
    return data, count, dictionary, reverse_dictionary

# build the datset
data, count, dictionary, reverse_dictionary = build_dataset(words)
# free up some memory
del words
# print out stats
print('most common words (+UNK):', count[:10])
print('sample data:', data[:10], [reverse_dictionary[i] for i in data[:10]])
vocabulary_size = len(dictionary)
print('Dictionary size', len(dictionary))

most common words (+UNK): [['UNK', 0], ('diabetes', 65794), ('patients', 19961), ('study', 19829), ("'s", 19400), ('type', 19066), ('health', 16383), ('risk', 15483), ('blood', 14819), ('disease', 14818)]
sample data: [24, 5281, 1129, 2226, 201, 571, 50, 3130, 5441, 6125] ['title', 'verichip', 'corporation', 'announces', 'phase', 'ii', 'development', 'vivo', 'glucose-sensing', 'rfid']
Dictionary size 66495


In [15]:
data_index = 0

# generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    # make sure our parameters are self-consistent
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    # create empty batch ndarray using 'batch_size'
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    # create empty labels ndarray using 'batch_size'
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    # [ skip_window target skip_window ]
    span = 2 * skip_window + 1
    # create a buffer object for prepping batch data
    buffer = collections.deque(maxlen=span)
    # for each element in our calculated span, append the datum at 'data_index' and increment 'data_index' moduli the amount of data
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # loop for 'batch_size' // 'num_skips'
    for i in range(batch_size // num_skips):
         # target label at the center of the buffer
        target = skip_window
        targets_to_avoid = [skip_window]
        # loop for 'num_skips'
        for j in range(num_skips):
            # loop through all 'targets_to_avoid'
            while target in targets_to_avoid:
                # pick a random index as target
                target = random.randint(0, span - 1)
            # put it in 'targets_to_avoid'
            targets_to_avoid.append(target)
            # set the skip window in the minibatch data
            batch[i * num_skips + j] = buffer[skip_window]
            # set the target in the minibatch labels
            labels[i * num_skips + j, 0] = buffer[target]
        # add the data at the current 'data_index' to the buffer
        buffer.append(data[data_index])
        # increment 'data_index'
        data_index = (data_index + 1) % len(data)
    # return the minibatch data and corresponding labels
    return batch, labels

# get a minibatch
batch, labels = generate_batch(batch_size=30, num_skips=2, skip_window=2)

# print out part of the minibatch to the console
for i in range(30):
    print(batch[i], reverse_dictionary[batch[i]], '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

1129 corporation -> 5281 verichip
1129 corporation -> 24 title
2226 announces -> 5281 verichip
2226 announces -> 1129 corporation
201 phase -> 571 ii
201 phase -> 50 development
571 ii -> 201 phase
571 ii -> 50 development
50 development -> 571 ii
50 development -> 201 phase
3130 vivo -> 50 development
3130 vivo -> 571 ii
5441 glucose-sensing -> 50 development
5441 glucose-sensing -> 6125 rfid
6125 rfid -> 3130 vivo
6125 rfid -> 1105 receptors
6029 microchip -> 1105 receptors
6029 microchip -> 6125 rfid
1105 receptors -> 2678 llc
1105 receptors -> 6125 rfid
2678 llc -> 5281 verichip
2678 llc -> 1105 receptors
5281 verichip -> 1105 receptors
5281 verichip -> 2678 llc
1129 corporation -> 5281 verichip
1129 corporation -> 2678 llc
5281 verichip -> 5281 verichip
5281 verichip -> 2227 nasdaq
2227 nasdaq -> 50 development
2227 nasdaq -> 4521 chip


In [16]:
 #hyperparameters
batch_size = 128
embedding_size = 128 # dimension of the embedding vector
skip_window = 2 # how many words to consider to left and right
num_skips = 2 # how many times to reuse an input to generate a label

# we choose random validation dataset to sample nearest neighbors
# here, we limit the validation samples to the words that have a low
# numeric ID, which are also the most frequently occurring words
valid_size = 16 # size of random set of words to evaluate similarity on
valid_window = 100 # only pick development samples from the first 'valid_window' words
valid_examples = np.random.choice(valid_window, valid_size, replace=False)
num_sampled = 64 # number of negative examples to sample

# create computation graph
graph = tf.Graph()

with graph.as_default():
    # input data
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    
    # operations and variables
    # look up embeddings for inputs
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, train_inputs)

    # construct the variables for the NCE loss
    nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    # compute the average NCE loss for the batch.
    # tf.nce_loss automatically draws a new sample of the negative labels each time we evaluate the loss.
    loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights, biases=nce_biases,
                     labels=train_labels, inputs=embed, num_sampled=num_sampled, num_classes=vocabulary_size))
    
    # construct the SGD optimizer using a learning rate of 1.0
    optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

    # compute the cosine similarity between minibatch examples and all embeddings
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

    # add variable initializer
    init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [17]:
# steps to train the model
num_steps = 500000

with tf.Session(graph=graph) as session:
    # we must initialize all variables before using them
    init.run()
    print('initialized.')
    
    # loop through all training steps and keep track of loss
    average_loss = 0
    for step in xrange(num_steps):
        # generate a minibatch of training data
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}
        
        # we perform a single update step by evaluating the optimizer operation (including it
        # in the list of returned values of session.run())
        _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
        average_loss += loss_val
        
        # print average loss every 2,000 steps
        if step % 2000 == 0:
            if step > 0:
                average_loss /= 2000
            # the average loss is an estimate of the loss over the last 2000 batches.
            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0
        
        # computing cosine similarity (expensive!)
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in xrange(valid_size):
                # get a single validation sample
                valid_word = reverse_dictionary[valid_examples[i]]
                # number of nearest neighbors
                top_k = 8
                # computing nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = "nearest to %s:" % valid_word
                for k in xrange(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = "%s %s," % (log_str, close_word)
                print(log_str)
        
    final_embeddings = normalized_embeddings.eval()

initialized.
Average loss at step  0 :  309.557037354
nearest to children: slippery, rheumatologist, undermine, confine, voice, longdaysin, farmington, tele-ophthalmology,
nearest to including: single-cell, jump-started, pac, -p., metallo, jcnn, mid-february, ipfk,
nearest to could: certifying, end-to-end, phosphates, colorful, wcn, hummus, 'india, encouragement,
nearest to american: identify/prevent/manage, chemocentryx, adage, maximum, sandrina, next-generation, marian, lipanthyl,
nearest to high: provider-reported, reassurance, tayebati, flaxseed, rdi, dryden, pad, obesogens,
nearest to women: disease-which, trillennium, who-idf, afflicts, beta-adrenergic, remediable, marta, androutsellis-theotokis,
nearest to one: nsaids, wadia, liquor, biotechnology, later-born, pollen.com, downwards, omrf,
nearest to development: tamar, vignette, re-populate, pureffect™, brüschweiler, difficult-to-study, usage, scratched,
nearest to drug: e.m., whites, nephrology, steatosis, insulin.studies, schu

Average loss at step  42000 :  5.87244332826
Average loss at step  44000 :  5.62861843741
Average loss at step  46000 :  5.59267273569
Average loss at step  48000 :  5.58443589556
Average loss at step  50000 :  5.43112347472
nearest to children: people, women, mec, overweight, iron-sulfur, oocytes, intercept, adults,
nearest to including: lcrmp-, include, related, ibanez, bioresearch, quitting, srt, montpellier,
nearest to could: may, interactome, might, nd-, certifying, montpellier, would, also,
nearest to american: chemocentryx, mustache, african, c-section, telling, identify/prevent/manage, gases, title,
nearest to high: higher, increase, elevated, increased, lower, flaxseed, low, unawareness,
nearest to women: pregnancy, children, symptoms, men, self-harm, patients, extensive, fluorescent,
nearest to one: two, olly, hla-dr-dq, montpellier, aadr, wii, year, nd-,
nearest to development: montpellier, dimension, nordisk, venkata, potential, retinitis, pio, end-of-life,
nearest to drug:

Average loss at step  92000 :  4.76231454575
Average loss at step  94000 :  4.75526247537
Average loss at step  96000 :  4.74156077361
Average loss at step  98000 :  4.74132520068
Average loss at step  100000 :  4.74135781109
nearest to children: people, women, adults, mec, overweight, child, undermine, iron-sulfur,
nearest to including: include, well, related, associated, lcrmp-, among, like, homicide,
nearest to could: may, might, would, interactome, treatments, jpec, also, hope,
nearest to american: mustache, chemocentryx, african, ada, compounds-, c-section, journal, international,
nearest to high: higher, elevated, low, lower, increased, increase, unawareness, unhealthy,
nearest to women: men, pregnancy, children, people, patients, participants, individuals, self-harm,
nearest to one: two, hla-dr-dq, aadr, every, many, three, first, olly,
nearest to development: montpellier, potential, end-of-life, dimension, novel, understanding, regulating, retinitis,
nearest to drug: drugs, med

Average loss at step  142000 :  4.54260649765
Average loss at step  144000 :  4.55059501255
Average loss at step  146000 :  4.56661237311
Average loss at step  148000 :  4.58463922143
Average loss at step  150000 :  4.57993861496
nearest to children: women, people, adults, child, individuals, diabetics, mec, offspring,
nearest to including: include, well, related, associated, like, various, among, lcrmp-,
nearest to could: may, might, would, potential, hope, interactome, jpec, potentially,
nearest to american: mustache, canadian, journal, ada, compounds-, international, african, entrenched,
nearest to high: higher, elevated, low, lower, increased, increase, unhealthy, provider-reported,
nearest to women: men, children, pregnancy, participants, patients, people, individuals, adults,
nearest to one: two, every, another, three, hla-dr-dq, example, aadr, first,
nearest to development: developing, discovery, end-of-life, montpellier, potential, novel, commercialization, regulating,
nearest 

Average loss at step  192000 :  4.435949265
Average loss at step  194000 :  4.47917505825
Average loss at step  196000 :  4.43848316729
Average loss at step  198000 :  4.43699276793
Average loss at step  200000 :  4.47254688489
nearest to children: women, people, child, adults, diabetics, individuals, person, jackie,
nearest to including: include, related, well, like, various, linked, associated, and/or,
nearest to could: may, might, would, hope, potential, interactome, potentially, lipocalin,
nearest to american: canadian, mustache, ada, journal, compounds-, international, endocrinologists, congrès,
nearest to high: higher, low, elevated, lower, increased, unhealthy, lipids, unawareness,
nearest to women: men, participants, people, children, pregnancy, patients, individuals, adults,
nearest to one: every, another, two, example, three, least, hla-dr-dq, many,
nearest to development: developing, discovery, develop, end-of-life, commercialization, treatment, prdm, targeting,
nearest to d

...

Average loss at step  292000 :  4.35135765839
Average loss at step  294000 :  4.35517165524
Average loss at step  296000 :  4.33571745455
Average loss at step  298000 :  4.33640482807
Average loss at step  300000 :  4.37295647252
nearest to children: women, child, people, adults, offspring, diabetics, older, overweight,
nearest to including: include, related, well, included, various, associated, like, linked,
nearest to could: may, might, would, hope, potential, potentially, ramnik, jpec,
nearest to american: canadian, rbcs, international, certified, aha, journal, messengers, americans,
nearest to high: higher, low, elevated, lower, increased, greater, unhealthy, increasing,
nearest to women: men, children, participants, people, individuals, pregnancy, patients, adults,
nearest to one: another, two, every, example, second, three, least, many,
nearest to development: develop, developing, targeting, discovery, commercialization, end-of-life, optimize, agenda,
nearest to drug: drugs, medi

Average loss at step  342000 :  4.3581393342
Average loss at step  344000 :  4.32915801132
Average loss at step  346000 :  4.30430566466
Average loss at step  348000 :  4.34168362331
Average loss at step  350000 :  4.3008417626
nearest to children: women, child, offspring, adults, people, persons, individuals, adolescents,
nearest to including: include, well, related, included, various, associated, like, linked,
nearest to could: may, might, would, hope, potentially, potential, ramnik, able,
nearest to american: canadian, aha, international, ada, rbcs, european, british, americans,
nearest to high: higher, elevated, low, lower, increased, lipids, highest, increasing,
nearest to women: men, children, individuals, people, participants, patients, pregnancy, adults,
nearest to one: another, two, every, example, second, however, three, least,
nearest to development: developing, develop, targeting, discovery, agenda, optimize, production, end-of-life,
nearest to drug: drugs, medication, mulè

Average loss at step  392000 :  4.31254230762
Average loss at step  394000 :  4.29655810678
Average loss at step  396000 :  4.27734209383
Average loss at step  398000 :  4.28873064792
Average loss at step  400000 :  4.26693641484
nearest to children: women, people, offspring, child, diabetics, adults, adolescents, kids,
nearest to including: include, related, well, included, various, includes, and/or, associated,
nearest to could: may, might, would, hope, potentially, potential, ramnik, twenty-six,
nearest to american: canadian, international, aha, european, journal, messengers, kaneda, ada,
nearest to high: higher, elevated, low, lower, increased, highest, greater, increases,
nearest to women: men, children, individuals, pregnancy, people, patients, participants, adults,
nearest to one: another, every, two, example, second, major, half, least,
nearest to development: developing, develop, commercialization, discovery, targeting, agenda, optimize, end-of-life,
nearest to drug: drugs, me

Average loss at step  442000 :  4.26717413151
Average loss at step  444000 :  4.23168505991
Average loss at step  446000 :  4.28264397931
Average loss at step  448000 :  4.25600473249
Average loss at step  450000 :  4.23108474827
nearest to children: women, child, people, diabetics, offspring, adults, persons, kids,
nearest to including: include, related, well, included, includes, various, and/or, like,
nearest to could: may, might, would, hope, potential, potentially, twenty-six, ramnik,
nearest to american: canadian, aha, international, messengers, glucose-control, journal, kaneda, compounds-,
nearest to high: higher, low, elevated, lower, increased, highest, greater, lowered,
nearest to women: men, children, people, pregnancy, adults, patients, mothers, individuals,
nearest to one: another, two, every, second, example, although, half, least,
nearest to development: developing, commercialization, develop, optimize, discovery, agenda, end-of-life, targeting,
nearest to drug: drugs, me

Average loss at step  492000 :  4.25858087182
Average loss at step  494000 :  4.27850131643
Average loss at step  496000 :  4.27891448832
Average loss at step  498000 :  4.2263347798
Average loss at step  500000 :  4.2360491209
nearest to children: women, child, offspring, infants, diabetics, kids, adults, woman,
nearest to including: include, related, well, included, associated, includes, and/or, addition,
nearest to could: may, might, would, hope, potential, potentially, ramnik, able,
nearest to american: canadian, aha, americans, messengers, ada, international, compounds-, glucose-control,
nearest to high: higher, elevated, low, lower, increased, highest, lowered, increasing,
nearest to women: men, pregnancy, children, mothers, patients, people, adults, individuals,
nearest to one: another, two, every, example, second, half, lipophilic, although,
nearest to development: discovery, developing, develop, optimize, commercialization, targeting, end-of-life, onset,
nearest to drug: drugs

In [18]:
def plot_with_labels(low_dim_embs, labels, filename='tsne.png'):
  assert low_dim_embs.shape[0] >= len(labels), 'More labels than embeddings'
  plt.figure(figsize=(18, 18))  # in inches
  for i, label in enumerate(labels):
    x, y = low_dim_embs[i, :]
    plt.scatter(x, y)
    plt.annotate(label,
                 xy=(x, y),
                 xytext=(5, 2),
                 textcoords='offset points',
                 ha='right',
                 va='bottom')

  plt.savefig(filename)

In [19]:
try:
  # pylint: disable=g-import-not-at-top
  from sklearn.manifold import TSNE
  import matplotlib.pyplot as plt

  tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, method='exact')
  plot_only = 500
  low_dim_embs = tsne.fit_transform(final_embeddings[:plot_only, :])
  labels = [reverse_dictionary[i] for i in xrange(plot_only)]
  plot_with_labels(low_dim_embs, labels)

except ImportError:
  print('Please install sklearn, matplotlib, and scipy to show embeddings.')